
# Homework 3 - Pandas and SQL

16.03.2022

Kriti Amin

# Pfam database

- Collection of protein domain families
- Clans of related protein families
- Contains 3 tables
 - clan
 - family
 - protein

- Documentation - https://pfam-docs.readthedocs.io/en/latest/index.html

*Note:* This is sample database created for the purpose of the exercise.

## Load database

In [5]:
# connect to database
connection = 'sqlite:///pfam.db'

In [6]:
%load_ext sql
%sql sqlite:///pfam.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


# Query tables with pandas from database

- Clan
- Family
- Protein

In [7]:
import pandas as pd

In [8]:
clan = pd.read_sql('clan', connection)
family = pd.read_sql('family', connection)
protein = pd.read_sql('protein', connection)

In [9]:
for table in [clan, family, protein]:
    if 'index' in table.columns:
        table.drop(columns='index', inplace=True)

# Table: Clan

- contains list of families in the clan

In [10]:
clan

,id,accession,description,families
0,KA1-like,CL0573,Kinase associated domain 1-like,"AdenylateSensor,KA1,NAF,NAM-associated"
1,Keratin_assoc,CL0520,Keratin_associated superfamily,"Keratin_B2,Keratin_B2_2,PMG"
2,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase","Misat_Tub_SegII,Tubulin,Tubulin_2,Tubulin_3"


### Split list of family ids from clan

In [11]:
clan['family_id'] = clan['families'].apply(lambda x: x.split(','))
clan.drop(columns='families', inplace=True)
clan

,id,accession,description,family_id
0,KA1-like,CL0573,Kinase associated domain 1-like,"[AdenylateSensor, KA1, NAF, NAM-associated]"
1,Keratin_assoc,CL0520,Keratin_associated superfamily,"[Keratin_B2, Keratin_B2_2, PMG]"
2,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase","[Misat_Tub_SegII, Tubulin, Tubulin_2, Tubulin_3]"


### Explode the list of family ids

In [12]:
clan_split = clan.explode(column='family_id')
clan_split

,id,accession,description,family_id
0,KA1-like,CL0573,Kinase associated domain 1-like,AdenylateSensor
0,KA1-like,CL0573,Kinase associated domain 1-like,KA1
0,KA1-like,CL0573,Kinase associated domain 1-like,NAF
0,KA1-like,CL0573,Kinase associated domain 1-like,NAM-associated
1,Keratin_assoc,CL0520,Keratin_associated superfamily,Keratin_B2
1,Keratin_assoc,CL0520,Keratin_associated superfamily,Keratin_B2_2
1,Keratin_assoc,CL0520,Keratin_associated superfamily,PMG
2,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase",Misat_Tub_SegII
2,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase",Tubulin
2,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase",Tubulin_2


# Map clan to family 

- List of family ids mapped to  family accession id in family table

In [13]:
family

,id,accession
0,Tubulin,PF00091
1,Tubulin_2,PF13809
2,Misat_Tub_SegII,PF10644
3,KA1,PF02149
4,AdenylateSensor,PF16579
5,Keratin_B2,PF01500
6,PMG,PF05287
7,Keratin_B2_2,PF13885
8,Tubulin_3,PF14881


### Join clan and family table through family id

In [14]:
clan_family = clan_split.set_index('family_id').join(family.set_index('id'), how='inner', lsuffix='_clan', rsuffix='_family')
clan_family

,id,accession_clan,description,accession_family
AdenylateSensor,KA1-like,CL0573,Kinase associated domain 1-like,PF16579
KA1,KA1-like,CL0573,Kinase associated domain 1-like,PF02149
Keratin_B2,Keratin_assoc,CL0520,Keratin_associated superfamily,PF01500
Keratin_B2_2,Keratin_assoc,CL0520,Keratin_associated superfamily,PF13885
PMG,Keratin_assoc,CL0520,Keratin_associated superfamily,PF05287
Misat_Tub_SegII,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase",PF10644
Tubulin,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase",PF00091
Tubulin_2,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase",PF13809
Tubulin_3,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase",PF14881


# Protein and clan

- Map protein entires from database to clan

In [15]:
protein

,id,accession,description,taxid,family_accession
0,X6P775_RETFI,X6P775,Tubulin alpha chain,46433,PF00091
1,W6XY45_COCCA,W6XY45,Gamma-tubulin,930089,PF00091
2,B7KI31_GLOC7,B7KI31,zinc_ribbon_2 domain-containing protein,65393,PF13809
3,V4L4I8_9GAMM,V4L4I8,VWFA domain-containing protein,1385625,PF13809
4,R9AA50_WALI9,R9AA50,Fumarate hydratase,1299270,PF10644
5,A0A556U8F7_BAGYA,A0A556U8F7,Maternal embryonic leucine zipper kinase,175774,PF02149
6,Q6RXY1_MAIZEQ6RXY1,Q6RXY1,Non-specific serine/threonine protein kinase,4577,PF02149
7,T1KCF8_TETURT1KCF8,T1KCF8,Protein kinase domain-containing protein,32264,PF16579
8,A0A6I9IMU7_VICPAA0A6I9IMU7,A0A6I9IMU7,keratin-associated protein 4-3-like,30538,PF01500
9,A0A6J0APB3_VICPAA0A6J0APB3,A0A6J0APB3,keratin-associated protein 10-1-like,30538,PF05287


### Join clan and protein table through family accession id

In [16]:
protein_clan = protein.set_index('family_accession').join(clan_family.set_index('accession_family'), how='left',\
                                                         lsuffix='_protein', rsuffix='_clan')
protein_clan.reset_index(inplace=True)
protein_clan.rename(columns={'index' : 'family_accession'}, inplace=True)
protein_clan

,family_accession,id_protein,accession,description_protein,taxid,id_clan,accession_clan,description_clan
0,PF00091,X6P775_RETFI,X6P775,Tubulin alpha chain,46433,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase"
1,PF00091,W6XY45_COCCA,W6XY45,Gamma-tubulin,930089,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase"
2,PF01500,A0A6I9IMU7_VICPAA0A6I9IMU7,A0A6I9IMU7,keratin-associated protein 4-3-like,30538,Keratin_assoc,CL0520,Keratin_associated superfamily
3,PF02149,A0A556U8F7_BAGYA,A0A556U8F7,Maternal embryonic leucine zipper kinase,175774,KA1-like,CL0573,Kinase associated domain 1-like
4,PF02149,Q6RXY1_MAIZEQ6RXY1,Q6RXY1,Non-specific serine/threonine protein kinase,4577,KA1-like,CL0573,Kinase associated domain 1-like
5,PF05287,A0A6J0APB3_VICPAA0A6J0APB3,A0A6J0APB3,keratin-associated protein 10-1-like,30538,Keratin_assoc,CL0520,Keratin_associated superfamily
6,PF05287,A0A7N5JVJ5_AILMEA0A7N5JVJ5,A0A7N5JVJ5,Uncharacterized protein,9646,Keratin_assoc,CL0520,Keratin_associated superfamily
7,PF10644,R9AA50_WALI9,R9AA50,Fumarate hydratase,1299270,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase"
8,PF13809,B7KI31_GLOC7,B7KI31,zinc_ribbon_2 domain-containing protein,65393,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase"
9,PF13809,V4L4I8_9GAMM,V4L4I8,VWFA domain-containing protein,1385625,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase"


# Search specific information

In [17]:
df = protein_clan[['id_protein', 'accession', 'description_protein', 'id_clan', 'accession_clan', 'description_clan']]
df

,id_protein,accession,description_protein,id_clan,accession_clan,description_clan
0,X6P775_RETFI,X6P775,Tubulin alpha chain,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase"
1,W6XY45_COCCA,W6XY45,Gamma-tubulin,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase"
2,A0A6I9IMU7_VICPAA0A6I9IMU7,A0A6I9IMU7,keratin-associated protein 4-3-like,Keratin_assoc,CL0520,Keratin_associated superfamily
3,A0A556U8F7_BAGYA,A0A556U8F7,Maternal embryonic leucine zipper kinase,KA1-like,CL0573,Kinase associated domain 1-like
4,Q6RXY1_MAIZEQ6RXY1,Q6RXY1,Non-specific serine/threonine protein kinase,KA1-like,CL0573,Kinase associated domain 1-like
5,A0A6J0APB3_VICPAA0A6J0APB3,A0A6J0APB3,keratin-associated protein 10-1-like,Keratin_assoc,CL0520,Keratin_associated superfamily
6,A0A7N5JVJ5_AILMEA0A7N5JVJ5,A0A7N5JVJ5,Uncharacterized protein,Keratin_assoc,CL0520,Keratin_associated superfamily
7,R9AA50_WALI9,R9AA50,Fumarate hydratase,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase"
8,B7KI31_GLOC7,B7KI31,zinc_ribbon_2 domain-containing protein,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase"
9,V4L4I8_9GAMM,V4L4I8,VWFA domain-containing protein,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase"


### Search proteins from one clan (e.g. 'Tubulin' )

In [18]:
df[df['id_clan'] == 'Tubulin']

,id_protein,accession,description_protein,id_clan,accession_clan,description_clan
0,X6P775_RETFI,X6P775,Tubulin alpha chain,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase"
1,W6XY45_COCCA,W6XY45,Gamma-tubulin,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase"
7,R9AA50_WALI9,R9AA50,Fumarate hydratase,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase"
8,B7KI31_GLOC7,B7KI31,zinc_ribbon_2 domain-containing protein,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase"
9,V4L4I8_9GAMM,V4L4I8,VWFA domain-containing protein,Tubulin,CL0566,"Tubulin nucleotide-binding domain-like, GTPase"


### Search clan of protein from  protein accession (e.g. 'A0A6I9IMU7' )

In [19]:
df[df['accession'] == 'A0A6I9IMU7']

,id_protein,accession,description_protein,id_clan,accession_clan,description_clan
2,A0A6I9IMU7_VICPAA0A6I9IMU7,A0A6I9IMU7,keratin-associated protein 4-3-like,Keratin_assoc,CL0520,Keratin_associated superfamily


# Conclusion

- link tables based on common data
- find specific information from database
- create new tables with only required information

# Thank You

# Create database

- Run following cells to create the pfam database used in this notebook.
- Change paths (if needed)

In [20]:
# import pandas as pd

# files_path = 'C:\\Users\\kriti\\Desktop\\BioDB\\2-kriti\\Homework\\Day_3\\Pfam\\'
# connection = 'sqlite:///pfam.db'

In [21]:
# %load_ext sql
# %sql sqlite:///pfam.db

In [22]:
# clan = pd.read_csv(file_path+'clan.csv')
# family = pd.read_csv(file_path+'family.csv')
# protein = pd.read_csv(file_path+'protein.csv')

In [23]:
# clan.to_sql('clan', connection, if_exists='replace')

# family.to_sql('family', connection, if_exists='replace')

# protein.to_sql('protein', connection, if_exists='replace')